In [1]:
# coding: utf-8

# In[2]:

import urllib.request
import requests
import zipfile
import io
import os
import glob
import pandas as pd
import boto
import boto.s3
import sys
import luigi
import datetime, time

import csv
import pandas as pd
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns

from boto.s3.key import Key
from urllib.request import urlopen
from bs4 import BeautifulSoup as bsoup

In [2]:
loanData = pd.read_csv('C:\\Users\\prith\\Documents\\Python Scripts\\ads assignment2\\Accepted_LoanData.csv', sep=',' , encoding='latin-1')

C:\Users\prith\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,19,49,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
loanData = loanData[pd.notnull(loanData['loan_amnt'])]

In [4]:
#delete columns which are of no use
del loanData['emp_title']
del loanData['desc']
del loanData['last_pymnt_d']
del loanData['next_pymnt_d']
del loanData['last_credit_pull_d']

In [5]:
#Converting to datatime format
loanData["issue_d"]=pd.to_datetime(loanData['issue_d'])
loanData["earliest_cr_line"]=pd.to_datetime(loanData['earliest_cr_line'])
#adding Issue_Month and Issue_Year columns
loanData['issue_month'] = loanData['issue_d'].dt.month
loanData['issue_year'] = loanData['issue_d'].dt.year

In [6]:
#Deriving Credit age for every customer
loanData["credit_age"]=loanData["issue_d"]-loanData["earliest_cr_line"]

In [7]:
loanData["num_il_tl"]=loanData["num_il_tl"].fillna(0)
loanData["num_rev_accts"]=loanData["num_rev_accts"].fillna(0)
loanData["Lending_Club_Interest"]=loanData["total_pymnt"]-loanData["total_pymnt_inv"]

#Format Interest_Rate
loanData["Interest_Rate"]=loanData["int_rate"].str.rstrip("%")
loanData['Interest_Rate'] = loanData['Interest_Rate'].convert_objects(convert_numeric=True)

#Modifying emp_length
loanData["derived_emp_length"]=loanData["emp_length"]
loanData.derived_emp_length[loanData.derived_emp_length=="10+ years"]="10"
loanData.derived_emp_length[loanData.derived_emp_length=="< 1 year"]="0"
loanData['derived_emp_length'] = loanData.derived_emp_length.str.replace(' years?' , '')

#Modifying title by personal
loanData['derived_title'] = loanData['title']
loanData['derived_title'] = loanData['derived_title'].fillna('personal')

#mths_since_last_delinq - replace by 24 if delinq_2yrs = 0
loanData['derived_mths_since_last_delinq'] = loanData['mths_since_last_delinq']
loanData['derived_mths_since_last_delinq'] = loanData['derived_mths_since_last_delinq'].fillna(24)

#mths_since_last_record - replace blanks by -1, which indicates that a borrower doesn't have any public record
loanData['derived_mths_since_last_record'] = loanData['mths_since_last_record']
loanData['derived_mths_since_last_record'] = loanData['derived_mths_since_last_record'].fillna(-1)

#annual_inc - there are 4 blank records, each doesn't have a employer so replacing it with zero
loanData['derived_annual_inc'] = loanData['annual_inc']
loanData['derived_annual_inc'] = loanData['derived_annual_inc'].fillna(0)
#delinq_2yrs - delinq_2yrs has the max value of 39. Whereas the delinq_2yrs is null for all people which does not meet the
# credit policy is not valid and who status is Not Verified. So we are taking the mean of all the 
loanData['derived_delinq_2yrs'] = loanData['delinq_2yrs']
loanData['delinq_2yrs'] = loanData['delinq_2yrs'].fillna(100)
#delinq_amnt - Replacing this by 0, because 99.98% of the values is zero. Negligible rows has some other values
loanData['derived_delinq_amnt'] = loanData['delinq_amnt']
loanData['derived_delinq_amnt'] = loanData['derived_delinq_amnt'].fillna(0)
#revol_util - Replaced it with the mean
loanData['derived_revol_util'] = loanData['revol_util']
loanData["derived_revol_util"] = loanData["derived_revol_util"].str.rstrip("%")
loanData['derived_revol_util'] = loanData['derived_revol_util'].convert_objects(convert_numeric=True)
loanData['derived_revol_util'] = loanData['derived_revol_util'].fillna((loanData['derived_revol_util'].mean()))
#pub_rec_bankruptcies - Replace this by the mean as all are non verified and does not meet the credit policy.
loanData['derived_pub_rec_bankruptcies'] = loanData['pub_rec_bankruptcies']
loanData['derived_pub_rec_bankruptcies'] = loanData['derived_pub_rec_bankruptcies'].fillna(loanData['derived_pub_rec_bankruptcies'].mean())
#tax_liens
loanData['derived_tax_liens'] = loanData['tax_liens']
loanData['derived_tax_liens'] = loanData['derived_tax_liens'].fillna(loanData['derived_tax_liens'].mean())
#interest_rate
loanData['derived_int_rate'] = loanData['int_rate']
loanData["derived_int_rate"] = loanData["derived_int_rate"].str.rstrip("%")
loanData['derived_int_rate'] = loanData['derived_int_rate'].convert_objects(convert_numeric=True)
#FICO
loanData["FICO"]=(loanData["last_fico_range_high"]+loanData["last_fico_range_low"])/2
loanData["FICO"] = loanData[['FICO']].convert_objects(convert_numeric=True).fillna(300)
#Total_Acc
loanData["total_acc"] = loanData[['total_acc']].convert_objects(convert_numeric=True).fillna(loanData["total_acc"].mean())
#Account open past 24 months
loanData["acc_open_past_24mths"] = loanData[['acc_open_past_24mths']].convert_objects(convert_numeric=True).fillna(loanData["acc_open_past_24mths"].mean())
#Taking only numeric values from term
loanData["Derived_term"]=loanData["term"].str.extract("(\d+) months",expand=True)
loanData["Derived_term"] = loanData[['Derived_term']].convert_objects(convert_numeric=True)

C:\Users\prith\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\prith\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\prith\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\prith\Anaconda3\lib\site-packages\ipykernel\__main__.py:40: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\prith\Anaconda3\

In [8]:
#percent_bc_gt_75
loanData['derived_percent_bc_gt_75'] = loanData['percent_bc_gt_75']
loanData['derived_percent_bc_gt_75'] = loanData['derived_percent_bc_gt_75'].fillna(0)
#num_tl_op_past_12m
loanData['derived_num_tl_op_past_12m'] = loanData['num_tl_op_past_12m']
loanData['derived_num_tl_op_past_12m'] = loanData['derived_num_tl_op_past_12m'].fillna(0)

In [9]:
loanData.to_csv("Cleaned_Loaned_Data.csv")

In [10]:
loanData["LoanApproved"]=1
loanData["LoanApproved"]=(loanData["LoanApproved"]-1).where(loanData["policy_code"]==2.0,loanData["LoanApproved"])

In [11]:
clusterData=loanData[loanData["LoanApproved"]==1]

In [12]:
clusterInputData=clusterData[["sub_grade","derived_int_rate","derived_annual_inc","issue_year","application_type","Derived_term","FICO","derived_emp_length","addr_state","total_acc","loan_amnt","verification_status","home_ownership","acc_open_past_24mths","revol_bal","derived_mths_since_last_delinq"]].copy()

In [13]:
clusterInputData.to_csv("ClusterInputData.csv")

In [14]:
rejectedData = pd.read_csv('C:\\Users\\prith\\Documents\\Python Scripts\\ads assignment2\\Declined_LoanData.csv', sep=',' , encoding='latin-1')

In [16]:
x100 = rejectedData[pd.notnull(rejectedData['Risk_Score'])]
x10=x100.groupby("State")["Risk_Score"].mean().reset_index(name="Risk_Score_mean")
x10['Risk_Score_mean'] = pd.to_numeric(x10['Risk_Score_mean'])

rejectedData["Employment Length Value"]=rejectedData["Employment Length"].str.extract("(\d+)",expand=True)
rejectedData['Employment Length Value'] = pd.to_numeric(rejectedData['Employment Length Value'])

rejectedData["DTI"]=rejectedData["Debt-To-Income Ratio"].str.rstrip("%")
rejectedData['DTI'] = pd.to_numeric(rejectedData['DTI'])

x11=rejectedData.groupby("State")["Employment Length Value"].mean().reset_index(name="Employment Length Value Mean")
x12=pd.merge(rejectedData,x10,how='left')
rejectedData=pd.merge(x12,x11,how='left')

rejectedData["Risk_Score"]=(rejectedData["Risk_Score_mean"]+43).where((rejectedData["Risk_Score"].isnull()) & (rejectedData["DTI"]>43),rejectedData["Risk_Score_mean"])

rejectedData["Risk_Score"]=(rejectedData["Risk_Score"]+93).where((rejectedData["Risk_Score"].isnull()) & (rejectedData["DTI"]>43) &(rejectedData["Employment Length Value"]<rejectedData["Employment Length Value Mean"]),rejectedData["Risk_Score_mean"])

rejectedData["Risk_Score"]=(rejectedData["Risk_Score_mean"]).where((rejectedData["Risk_Score"].isnull()),rejectedData["Risk_Score_mean"])

rejectedData['Loan Title'] = rejectedData['Loan Title'].fillna('Others')

rejectedData['State'] = rejectedData[['State']].convert_objects(convert_numeric=True).fillna('Other')

rejectedData['Zip Code'] = rejectedData[['Zip Code']].convert_objects(convert_numeric=True).fillna('000xx')

#Converting to datatime format
rejectedData["Application Date"]=pd.to_datetime(rejectedData['Application Date'])
rejectedData['Application Date Month'] = rejectedData['Application Date'].dt.month
rejectedData['Application Date Year'] = rejectedData['Application Date'].dt.year

C:\Users\prith\Anaconda3\lib\site-packages\ipykernel\__main__.py:23: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\prith\Anaconda3\lib\site-packages\ipykernel\__main__.py:25: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [17]:
data3=rejectedData[["Amount Requested","Loan Title","Risk_Score","Debt-To-Income Ratio","Zip Code","Employment Length","State","Policy Code"]].copy()
data3["LoanApproved"]=0
data3 = data3.rename(columns={'Amount Requested': 'loan_amnt', 'Loan Title': 'title', 'Risk_Score': 'FicoScore', 'Debt-To-Income Ratio': 'dti', 'Zip Code': 'zip_code', 'Employment Length': 'emp_length', 'State': 'addr_state', 'Policy Code': 'policy_code', 'LoanApproved': 'LoanApproved'})

In [18]:
bigdata = loanData.append(data3, ignore_index=True)

In [19]:
bigdata.to_csv('Classification_Input_Data.csv')